In [ ]:
import pandas as pd
import os
import google.generativeai as genai
import numpy as np
import textwrap
import requests
import nltk

from IPython.display import display
from IPython.display import Markdown
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.stats import pearsonr
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer

#Aumentar a visualização
pd.set_option('display.max_colwidth', None)


In [2]:
#Ler arquivo csv
data = pd.read_csv('dataset_rec.csv')

In [3]:
#Visualizar melhor a primeira linha para entender seus componentes
print(data.iloc[0])

url                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

<b>Primeira estratégia: </b>
<br>Utilizar o embedding já disponibilizado na base de dados

In [4]:
# Função para converter a string do embedding para uma lista
def string_to_embedding(embedding_str):
    embedding_str = embedding_str.strip("[]").replace(" ", "")
    numbers_str = embedding_str.split(",")
    numbers_float = [float(num) for num in numbers_str]
    return numbers_float

data['embedding'] = data['embedding'].apply(string_to_embedding)


In [5]:
# Funções para calcular diferentes métricas de similaridade para ver qual é a mais assertiva

def calcular_similaridade_cosseno(index1, index2):
    embedding1 = np.array(data['embedding'][index1]).reshape(1, -1)
    embedding2 = np.array(data['embedding'][index2]).reshape(1, -1)
    similarity = cosine_similarity(embedding1, embedding2)[0][0]
    return similarity

def calcular_similaridade_euclidiana(index1, index2):
    embedding1 = np.array(data['embedding'][index1]).reshape(1, -1)
    embedding2 = np.array(data['embedding'][index2]).reshape(1, -1)
    distancia = euclidean_distances(embedding1, embedding2)[0][0]
    similaridade = 1 / (1 + distancia) 
    return similaridade

def calcular_similaridade_pearson(index1, index2):
    embedding1 = data['embedding'][index1]
    embedding2 = data['embedding'][index2]
    correlation, _ = pearsonr(embedding1, embedding2)
    return correlation

# Lista de métricas de similaridade
metricas = {
    "Cosseno": calcular_similaridade_cosseno,
    "Euclidiana": calcular_similaridade_euclidiana,
    "Pearson": calcular_similaridade_pearson
}

In [6]:
# Testar para um número menor de matérias para visualizar os resultados
num_materias = 5

# Loop para cada matéria
for i in range(num_materias):
    materia_alvo = i
    titulo_materia_alvo = data['title'][i]

    print(f"\nMatéria Alvo: {titulo_materia_alvo} (Índice: {materia_alvo})")

    # Loop para cada métrica de similaridade
    for nome_metrica, funcao_similaridade in metricas.items():
        matriz_similaridade = np.zeros((num_materias, num_materias))
        for j in range(num_materias):
            matriz_similaridade[i, j] = funcao_similaridade(i, j)

        # Encontrar as matérias mais similares à matéria alvo
        similaridades = matriz_similaridade[materia_alvo]
        indices_ordenados = np.argsort(similaridades)[::-1]  
        materias_mais_similares = indices_ordenados[1:11]  

        print(f"\nMétrica de similaridade: {nome_metrica}:")
        print("Títulos das matérias mais similares:")

        # Ver os títulos das matérias mais similares
        for indice in materias_mais_similares:
            print(f"- {data['title'][indice]}")



Matéria Alvo: Maira Cardi ameaça expor 'a verdade' sobre história com mulher de Thiago Silva: 'Não me desafie' (Índice: 0)

Métrica de similaridade: Cosseno:
Títulos das matérias mais similares:
- Mãe de Larissa Manoela, Silvana Taques diz em entrevista: 'Nunca quisemos ficar com nada dela'
- Larissa Manoela tem dia de malhação com o noivo André Luiz Frambach
- Mileide Mihaile mostra look para show de Beyoncé em Miami e realiza sonho como fã
- Dublador de Mario se aposenta de personagem após mais de 30 anos

Métrica de similaridade: Euclidiana:
Títulos das matérias mais similares:
- Mãe de Larissa Manoela, Silvana Taques diz em entrevista: 'Nunca quisemos ficar com nada dela'
- Larissa Manoela tem dia de malhação com o noivo André Luiz Frambach
- Mileide Mihaile mostra look para show de Beyoncé em Miami e realiza sonho como fã
- Dublador de Mario se aposenta de personagem após mais de 30 anos

Métrica de similaridade: Pearson:
Títulos das matérias mais similares:
- Mãe de Larissa Mano

Como os resultados não foram tão satisfatórios em nenhuma das métricas.
Uma materia sobre Maira Cardi deveria ter mais correlação com outras matérias sobre ela e isso não aconteceu.

<b>Segunda estratégia: </b>
<br>
Refazer o embeddings utilizando o TF_IDF e fazer a similaridade com a função cosseno.

In [7]:
data = data.drop(columns=['embedding'])

In [8]:
# Função para extrair o conteúdo de uma página web
def extrair_conteudo_web(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        conteudo = ' '.join([p.text for p in soup.find_all('p')])

        # Pegar apenas o conteúdo até +gshow
        conteudo = conteudo.split("+gshow")[0]

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
        return None
    except Exception as e:
        print(f"Erro ao processar o conteúdo da URL: {e}")
        return None

    return conteudo  

In [9]:
# Aplica a função para criar uma nova coluna com o conteúdo das matérias
data['conteudo'] = data['url'].apply(extrair_conteudo_web)

In [10]:
#Visualizar a coluna conteudo
print(data.iloc[0])

url                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [11]:
# Baixar stopwords do NLTK
nltk.download("stopwords")

# Definir stopwords em português
stop_words = stopwords.words("portuguese")

# Criar o modelo TF-IDF com stopwords em português
vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(data["conteudo"])


[nltk_data] Downloading package stopwords to C:\Users\Camila
[nltk_data]     Sajnin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Calcular a similaridade entre os textos utilizando a função cosseno
matriz_similaridade_tfidf = cosine_similarity(tfidf_matrix)

# Criar uma lista para armazenar as recomendações
recomendacoes = []

# Gerar recomendações para cada matéria
for idx in range(len(data)):
    similaridades = matriz_similaridade_tfidf[idx]  # Pega as similaridades da matéria atual
    indices_similares = similaridades.argsort()[::-1][1:11]  # Pega os 10 mais similares 

    # Pegar as URLs das matérias recomendadas
    recomendacoes.append([data.iloc[i]["url"] for i in indices_similares])

# Criar um DataFrame com as recomendações
colunas_recomendacoes = [f"recomendacao{i+1}" for i in range(10)]
df_recomendacoes = pd.DataFrame(recomendacoes, columns=colunas_recomendacoes)

# Juntar com os dados originais
df_recomendacoes_tf_idf = pd.concat([data[["url", "title"]], df_recomendacoes], axis=1)


display(df_recomendacoes_tf_idf)  

,url,title,recomendacao1,recomendacao2,recomendacao3,recomendacao4,recomendacao5,recomendacao6,recomendacao7,recomendacao8,recomendacao9,recomendacao10
0,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml,Maira Cardi ameaça expor 'a verdade' sobre história com mulher de Thiago Silva: 'Não me desafie',http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/belle-silva-se-pronuncia-sobre-rumores-de-thiago-participar-em-programa-de-emagrecimento.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-comenta-polemicas-e-desabafa-em-live.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-rebate-critica-de-daniel-cady.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-zilu-camargo-e-mais-veja-famosos-que-viraram-coach.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-fala-sobre-o-rompimento-com-a-filha.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/pipoca-da-ivete-volta-em-2a-temporada-com-novos-quadros-e-convidados.ghtml,http://gshow.globo.com/tudo-mais/pop/noticia/laura-cardoso-recebe-abraco-apertado-de-ingrid-guimaraes-no-festival-de-gramado.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/sabrina-sato-e-marcelo-adnet-participam-do-lancamento-de-sobre-nos-dois.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-da-sua-versao-da-disputa-com-a-filha-em-nova-entrevista.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml
1,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,Larissa Manoela tem dia de malhação com o noivo André Luiz Frambach,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-da-sua-versao-da-disputa-com-a-filha-em-nova-entrevista.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/filho-de-murilo-benicio-e-alessandra-negrini-antonio-benicio-anima-seguidores-com-foto-sem-camisa.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/familia-de-neymar-celebra-anuncio-do-jogador-no-al-hilal-estamos-juntos.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/gil-do-vigor-tem-carro-enguicado-ao-fazer-confusao-na-hora-de-abastecer-veja-video.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/anitta-comenta-affair-com-simone-susinna-pra-sempre-e-muito-forte.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/nasce-filha-de-bruno-do-klb-em-sao-paulo.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/felipe-neto-sai-em-defesa-de-luisa-sonza-apos-criticas-sobre-nova-musica.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/lea-garcia-estava-feliz-com-o-reconhecimento-e-falou-sobre-viver-por-mais-90-anos-em-ultima-entrevista.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/hariany-confirma-romance-com-filho-de-leonardo-estamos-ficando-nos-curtindo-esta-leve.ghtml
2,http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml,Dublador de Mario se aposenta de personagem após mais de 30 anos,http://gshow.globo.com/tudo-mais/pop/noticia/matheus-nachtergaele-sobre-o-auto-da-compadecida-2-nao-sei-o-tamanho-da-emocao-que-estou-sentindo.ghtml,http://gshow.globo.com/tudo-mais/pop/noticia/laura-cardoso-recebe-abraco-apertado-de-ingrid-guimaraes-no-festival-de-gramado.ghtml,http://gshow.globo.com/tudo-mais/pop/noticia/globoplay-lanca-3a-colecao-de-nfts-do-xuxaverso-com-venda-de-10-figurinhas-ineditas.ghtml,http://gshow.globo.com/tudo-mais/viralizou/noticia/post-de-anuncio-de-neymar-no-ah-hilal-ultrapassa-as-2-milhoes-de-curtidas-em-1-hora.ght

In [ ]:
#Salvar resultado
resultado.to_csv("materias_recomendadas.csv", index=False)

Os resultados usando o TF_IDF e similaridade cosseno foram mais satisfatórios. Apesar de ainda não estarem perfeitos. A segunda noticia da Larissa Manoela deveria se relacionar com as outras sobre a atriz, relaciona algumas, mas não todas.

<b>Terceira Alternativa:</b>
<br>
Utilizar a biblioteca sentence_transformer (SBERT) do hugging face, ao invés do TF_IDF e continuar usando a similaridade cosseno.

É necessário muita atenção ao utilizar bibliotecas terceiras, para não ter vazamento de dados.
<br>
Consultando a documentação dessa biblioteca é possível observar que ela roda localmente e não envia os dados para nenhuma fonte externa.
<br>
O código está disponível no github https://github.com/UKPLab/sentence-transformers

In [17]:
# Converter a coluna 'conteudo' para uma lista de strings
sentencas = data['conteudo'].tolist()

# Inicializar o modelo de embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Gerar embeddings para cada conteúdo
sentencas_embeddings = model.encode(sentencas)

# Calcular a matriz de similaridade coseno
matriz_similaridade_sbert = cosine_similarity(sentencas_embeddings)

# Criar um DataFrame de similaridade
df_similaridade_sbert = pd.DataFrame(matriz_similaridade_sbert, index=data['url'], columns=data['url'])
 
display(df_similaridade_sbert)

url,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mileide-mihaile-mostra-look-para-show-de-beyonce-em-miami-e-realiza-sonho-como-fa.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,http://gshow.globo.com/novelas/fuzue/vem-por-ai/noticia/fuzue-luna-descobre-pista-importante-sobre-o-passado-de-sua-mae-em-paraty.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/claudia-ohana-faz-book-com-roupa-estilosa-e-seguidores-elogiam-deusa-suprema.ghtml,http://gshow.globo.com/tudo-mais/pop/noticia/laura-cardoso-recebe-abraco-apertado-de-ingrid-guimaraes-no-festival-de-gramado.ghtml,http://gshow.globo.com/tudo-mais/viralizou/noticia/anitta-repreende-fa-que-estaria-cobrando-dinheiro-do-publico-na-hora-da-foto-veja-video.ghtml,http://gshow.globo.com/novelas/fuzue/noticia/fuzue-memes-sobre-a-trama-ganham-redes-sociais-na-primeira-semana.ghtml,...,http://gshow.globo.com/ep/loreal-paris/noticia/em-nova-fase-carla-diaz-mantem-rotina-de-cuidados.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/zagallo-apresenta-melhora-clinica-em-hospital-diz-boletim-medico.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/publico-compara-filha-com-personagem-de-gabriela-duarte-em-foto-publicada-pela-atriz.ghtml,http://gshow.globo.com/novelas/terra-e-paixao/vem-por-ai/noticia/terra-e-paixao-irene-da-tapa-na-cara-de-angelina-por-causa-de-agatha.ghtml,http://gshow.globo.com/novelas/terra-e-paixao/noticia/irene-vibra-com-noticia-morte-de-agatha-em-terra-e-paixao.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/nasce-filha-de-bruno-do-klb-em-sao-paulo.ghtml,http://gshow.globo.com/novelas/fuzue/resumo/20230828-a-20230903-4027771c62194aa1bc631806fe20aeef.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-rebate-critica-de-daniel-cady.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/veja-atores-que-sao-irmaos-na-vida-real-e-tambem-na-ficcao.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/filho-de-murilo-benicio-e-alessandra-negrini-antonio-benicio-anima-seguidores-com-foto-sem-camisa.ghtml
url,,,,,,,,,,,,,,,,,,,,,
http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml,1.000000,0.621516,0.332730,0.410857,0.658509,0.442035,0.538025,0.519603,0.568500,0.422391,...,0.479871,0.453743,0.562299,0.491495,0.459636,0.427552,0.488387,0.627561,0.434712,0.636085
http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,0.621516,1.000000,0.371875,0.428077,0.782795,0.493166,0.569897,0.564879,0.522334,0.489730,...,0.506677,0.527256,0.572915,0.452515,0.443974,0.420128,0.532405,0.579275,0.467033,0.697255
http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml,0.332730,0.371875,1.000000,0.324078,0.334699,0.322492,0.285612,0.412924,0.375968,0.387442,...,0.295922,0.232942,0.432433,0.390655,0.359585,0.345019,0.371174,0.420396,0.345408,0.457005
http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mileide-mihaile-mostra-look-para-show-de-beyonce-em-miami-e-realiza-sonho-como-fa.ghtml,0.410857,0.428077,0.324078,1.000000,0.491082,0.512790,0.582310,0.473214,0.579303,0.476595,...,0.468054,0.320615,0.544594,0.477818,0.433440,0.483662,0.478009,0.465686,0.355639,0.452302
http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,0.658509,0.7827

In [18]:
# Criar um DataFrame vazio para armazenar as recomendações
df_recomendacoes_sbert = data[["url", "title"]].copy()

# Definir as novas colunas de recomendação no df_recomendacoes_sbert
for i in range(1, 11):
    df_recomendacoes_sbert[f'recomendacao{i}'] = None

# Agora, para cada URL, encontramos as 10 mais similares
for idx, row in data.iterrows():
    # Obter a linha correspondente na matriz de similaridade (onde o índice é a URL atual)
    similaridades = df_similaridade.loc[row['url']].sort_values(ascending=False)
    
    # Pegar as 10 URLs mais similares, excluindo a URL própria
    recomendacoes = similaridades.drop(row['url']).head(10).index.tolist()
    
    # Preencher as colunas de recomendação
    for i, recomendacao_url in enumerate(recomendacoes, 1):
        df_recomendacoes_sbert.loc[idx, f'recomendacao{i}'] = recomendacao_url

# Exibir o DataFrame final com as recomendações
display(df_recomendacoes_sbert)

,url,title,recomendacao1,recomendacao2,recomendacao3,recomendacao4,recomendacao5,recomendacao6,recomendacao7,recomendacao8,recomendacao9,recomendacao10
0,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml,Maira Cardi ameaça expor 'a verdade' sobre história com mulher de Thiago Silva: 'Não me desafie',http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/belle-silva-se-pronuncia-sobre-rumores-de-thiago-participar-em-programa-de-emagrecimento.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-comenta-polemicas-e-desabafa-em-live.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-rebate-critica-de-daniel-cady.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,http://gshow.globo.com/realities/no-limite/2023/noticia/em-entrevista-exclusiva-eliminada-por-voto-aberto-fala-sobre-traicao-no-no-limite.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/deborah-secco-se-diverte-com-revelacao-intima-sobre-hugo-moura-exposto-em-rede-nacional.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-zilu-camargo-e-mais-veja-famosos-que-viraram-coach.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/karoline-lima-mostra-filha-com-eder-militao-falando-papai-no-dia-em-que-jogador-passa-por-cirurgia-no-joelho.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/deborah-secco-revela-detalhe-da-anatomia-intima-de-hugo-moura-na-estreia-de-sobre-nos-dois.ghtml,http://gshow.globo.com/novelas/terra-e-paixao/noticia/terra-e-paixao-debora-ozorio-detalha-estudo-sobre-abuso-sexual-para-viver-drama-de-petra.ghtml
1,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,Larissa Manoela tem dia de malhação com o noivo André Luiz Frambach,http://gshow.globo.com/novelas/mundo-de-novela/noticia/relembre-a-carreira-de-larissa-manoela.ghtml,http://gshow.globo.com/tudo-mais/pop/noticia/laura-cardoso-recebe-abraco-apertado-de-ingrid-guimaraes-no-festival-de-gramado.ghtml,http://gshow.globo.com/novelas/amor-perfeito/noticia/orlando-almeida-comenta-casamento-de-orlando-e-gilda-em-amor-perfeito-e-derrocada-da-vila-esta-roda-vai-girar.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/filho-de-murilo-benicio-e-alessandra-negrini-antonio-benicio-anima-seguidores-com-foto-sem-camisa.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/cezar-black-troca-beijos-com-morena-em-festival-de-musica-em-sao-paulo.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/elenco-de-terra-e-paixao-curte-noite-de-samba-ao-lado-de-viviane-araujo.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/belle-silva-se-pronuncia-sobre-rumores-de-thiago-participar-em-programa-de-emagrecimento.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/sheron-menezzes-enaltece-relacionamento-de-12-anos-com-marido-ele-me-completa.ghtml,http://gshow.globo.com/novelas/amor-perfeito/vem-por-ai/noticia/amor-perfeito-julio-encontra-personagem-que-sera-peca-chave-em-virada-da-novela.ghtml
2,http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml,Dublador de Mario se aposenta de personagem após mais de 30 anos,http://gshow.globo.com/novelas/amor-perfeito/vem-por-ai/noticia/amor-perfeito-julio-encontra-personagem-que-sera-peca-chave-em-virada-da-novela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/helena-ranaldi-longe-das-novelas-ha-9-anos-se-dedica-ao-teatro-e-nega-que-esteja-vivendo-em-sitio.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/sheron-menezzes-enaltece-relacionamento-de-12-anos-com-marido-ele-me-co

Os resultados ficaram piores que utilizando o TF_IDF. Essa biblioteca do hugging face não deve entender tão bem o português.
A segunda noticia sobre a Larissa Manoela não foi relacionada a outras noticias da atriz.

<b>Quarta Alternativa:</b>
<br>
Utilizar IA Generativa Google Gemini 1.5 Pro

In [19]:
#Credenciais google
load_dotenv(dotenv_path='.env')  
api_key = os.environ.get("GOOGLE_API_KEY")

genai.configure(api_key=api_key)

#Modelo utilizado
model = genai.GenerativeModel('gemini-1.5-pro-latest')

Para o primeiro teste quis avaliar se a IA Gemini compreendia bem português e se iria fazer as correlações corretas, então pedi para deixar bem detalhado a nota de similaridade dada e a justificativa

In [20]:
#Juntar informações dos Titulos
titulos = []
for i in data['title']:
    titulos.append(i)

In [21]:
#Juntar informações dos conteúdos
conteudo = []
for i in data['conteudo']:
    conteudo.append(i)

In [22]:
# Função para obter a avaliação de similaridade do Gemini
def avaliar_similaridade_gemini(titulo_alvo,materia_alvo):
    prompt = f"""
    Você é um especialista em análise de similaridade de textos. 
    Sua tarefa é avaliar o quão similares são os conteudos {conteudo} e me indicar as 10 matérias mais similares a {materia_alvo}

    Forneça uma pontuação de similaridade de 0 a 10, onde 0 significa que os títulos não têm nada em comum e 10 significa que os títulos tem muitos pontos em comum.

    Além da pontuação, forneça uma breve justificativa para sua avaliação (máximo de 2 frases).

    Formate sua resposta da seguinte forma:
    Matéria Alvo: {titulo_alvo}
    Matérias mais similares
    (1) Titulo da Materia - consulte {titulos}
    (1) Pontuação de similaridade: 
    (1) Justificativa:

    Por exemplo:
    Matéria Alvo: Mãe de Larissa Manoela, Silvana Taques diz em entrevista: 'Nunca quisemos ficar com nada dela'
    Matérias mais similares
    (1) Titulo da Materia:Mãe de Larissa Manoela diz que filha podia comprar o que quisesse: Em qualquer valor, inclusive carros
    (1) Pontuação de similaridade:10
    (1) Justificativa: Os dois textos falam sobre a relação da mãe de Larissa Manoela e a filha
    (2) Titulo da Materia: Mãe de Larissa Manoela responde sobre transferência de milhões da conta da filha e xingamento em mensagem
    (2) Pontuação de similaridade:9
    (2) Justificativa: Os dois textos falam sobre a mãe de Larissa Manoela 

    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Ocorreu um erro {e}")
        return None

In [ ]:
f'''
# Salvar resultados em um arquivo txt
nome_arquivo = "teste_materias_recomendadas_iagen.txt"
with open(nome_arquivo, "w", encoding="utf-8") as arquivo:
    for titulo in data['title']:
        resposta = avaliar_similaridade_gemini(titulo,data['conteudo'])
        arquivo.write(f"\n{resposta}\n")
        arquivo.write("-" * 50 + "\n") 
'''

Os resultados do teste foram bons e estão salvos no arquivo teste_materias_recomendadas_iagen.txt

Para fazer em larga escala, é impossível utilizar todo o conteudo por conta do custo, para otimizar podemos gerar um resumo

In [23]:
def gerar_resumo_conteudos(conteudo):
    prompt = f"""
    Você é um especialista em textos em português brasileiro. 
    Sua tarefa é gerar um resumo conciso do {conteudo} fornecido. 
    O resumo deve ser baseado nas palavras-chave mais relevantes do texto. 
    Retorne apenas as palavras-chave mais importantes para análise de similaridade.
    Não precisa de apresentações nem contexto adicional.
    Apenas forneça o resumo.
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Ocorreu um erro {e}")
        return None

In [24]:
# Criar um dicionario para armazenar os resumos e as urls
resumos_url = {}

# Gerar os resumos para cada matéria
for idx, url_atual in enumerate(data['url']):
    conteudo_atual = data['conteudo'][idx]  # Conteúdo correspondente à URL
    resumo = gerar_resumo_conteudos(conteudo_atual).strip()  # Removendo espaços extras e \n
    resumos_url[url_atual] = resumo  # Adicionar ao dicionário

# Adicionar a coluna de resumos ao DataFrame
data['resumo'] = data['url'].map(resumos_url)

# Exibir o DataFrame atualizado
display(data)

url  \
0                              http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml   
1                                 http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml   
2                                             http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml   
3                   http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mileide-mihaile-mostra-look-para-show-de-beyonce-em-miami-e-realiza-sonho-como-fa.ghtml   
4          http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml   
..                                                                                                                                                            ...   
95                                                           http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/nasce-filha-de-bruno-do-klb-em-sao-paulo.ghtml   
96                                                         http://gshow.globo.com/novelas/fuzue/resumo/20230828-a-20230903-4027771c62194aa1bc631806fe20aeef.ghtml   
97                                                          http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-rebate-critica-de-daniel-cady.ghtml   
98                                         http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/veja-atores-que-sao-irmaos-na-vida-real-e-tambem-na-ficcao.ghtml   
99  http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/filho-de-murilo-benicio-e-alessandra-negrini-antonio-benicio-anima-seguidores-com-foto-sem-camisa.ghtml   

                                                                                                 title  \
0     Maira Cardi ameaça expor 'a verdade' sobre história com mulher de Thiago Silva: 'Não me desafie'   
1                                  Larissa Manoela tem dia de malhação com o noivo André Luiz Frambach   
2                                     Dublador de Mario se aposenta de personagem após mais de 30 anos   
3                    Mileide Mihaile mostra look para show de Beyoncé em Miami e realiza sonho como fã   
4       Mãe de Larissa Manoela, Silvana Taques diz em entrevista: 'Nunca quisemos ficar com nada dela'   
..                                                                                                 ...   
95                                                          Nasce filha de Bruno, do KLB, em São Paulo   
96                                  Resumo de Fuzuê: veja os capítulos de 28 de agosto a 2 de setembro   
97                                  Maíra Cardi x Daniel Cady: entenda a troca de farpas entre os dois   
98                                          Veja atores que são irmãos na vida real e também na ficção   
99  Filho de Murilo Benício e Alessandra Negrini, Antonio Benício anima seguidores com foto sem camisa   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [25]:
# Exibir os primeiros 5 itens do dicionário
for url, resumo in list(resumos_url.items())[:5]:
    print(f"URL: {url}\nResumo: {resumo}\n{'-'*80}")


URL: http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml
Resumo: Maíra Cardi, Thiago Silva, Belle Silva, emagrecimento, programa, prints, conversas, mentira, desmente, treta, áudios, Daniel Cady, Ivete Sangalo, famosos, Ludmilla, Anitta, Larissa Manoela, seleção brasileira, Instagram, live, vídeos, desabafo.
--------------------------------------------------------------------------------
URL: http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml
Resumo: Larissa Manoela, André Luiz Frambach, malhação, rotina, Fantástico, entrevista, pais, rompimento financeiro, filme, Tá Escrito.
--------------------------------------------------------------------------------
URL: http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml
Resumo: Charles Martinet, Mario, Nintendo, dublador, Embaix

In [26]:
def gerar_recomendacoes_gemini(resumo_alvo,resumos):
    prompt = f"""
    Você é um especialista em análise de similaridade de textos. 
    Sua tarefa comparar matéria alvo com os resumos das outras matérias e retorne apenas as 10 URLs mais similares.
    Importante, SEMPRE retornar 10 matérias. As matérias não podem ser iguais entre si nem repetir a matéria que quero comparar.
    Para isso consulte os resumos de todas as materias em {resumos} e compare com {resumo_alvo}

    Forneça uma pontuação de similaridade de 0 a 10, onde 0 significa que os títulos não têm nada em comum e 10 significa que os títulos tem muitos pontos em comum.
    A partir dessa pontuação determine as 10 matérias mais similares.

    Retorne apenas as URLs separadas por "|", sem explicação adicional.
    Exemplo de resposta:
    url1 | url2 | url3 | url4 | url5 | url6 | url7 | url8 | url9 | url10

    """


    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Ocorreu um erro {e}")
        return None


In [27]:
# Criar lista para armazenar os dados formatados
dados_recomendacoes = []

# Gerar recomendações para cada matéria
for idx, url_alvo in enumerate(data['url']):
    titulo_alvo = data['title'][idx]
    resumo_alvo = data['resumo'][idx]
    urls_recomendadas = gerar_recomendacoes_gemini(resumo_alvo, resumos_url)

    # Garantir que a IA retornou recomendações no formato esperado
    if urls_recomendadas:
        recomendacoes_lista = urls_recomendadas.split(" | ")
    else:
        print("Erro no output da IA")

    # Adicionar a linha no formato correto
    dados_recomendacoes.append([url_alvo, titulo_alvo] + recomendacoes_lista)

# Criar um DataFrame com os resultados
colunas = ["url", "titulo"] + [f"recomendacao{i+1}" for i in range(10)]
df_recomendacoes_gemini = pd.DataFrame(dados_recomendacoes, columns=colunas)

display(df_recomendacoes_gemini)

,url,titulo,recomendacao1,recomendacao2,recomendacao3,recomendacao4,recomendacao5,recomendacao6,recomendacao7,recomendacao8,recomendacao9,recomendacao10
0,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml,Maira Cardi ameaça expor 'a verdade' sobre história com mulher de Thiago Silva: 'Não me desafie',http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/belle-silva-se-pronuncia-sobre-rumores-de-thiago-participar-em-programa-de-emagrecimento.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-rebate-critica-de-daniel-cady.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-comenta-polemicas-e-desabafa-em-live.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/famosos-coach-maira-cardi-zilu-camargo-leo-chaves-rodrigao-carol-dias-rafa-brites-charlie-sheen-guilherme-berenguer-emagrecimento-autoajuda-educacao-financas-palestras-desenvolvimento-pessoal.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-fala-sobre-o-rompimento-com-a-filha.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-da-sua-versao-da-disputa-com-a-filha-em-nova-entrevista.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/relembre-a-carreira-de-larissa-manoela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-ganhou-mais-de-2-milhoes-de-seguidores-apos-entrevista-sobre-o-rompimento-com-os-pais.ghtml\n
1,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,Larissa Manoela tem dia de malhação com o noivo André Luiz Frambach,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-taques-diz-em-entrevista-nunca-quisemos-ficar-com-nada-dela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-da-sua-versao-da-disputa-com-a-filha-em-nova-entrevista.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/mae-de-larissa-manoela-silvana-fala-sobre-o-rompimento-com-a-filha.ghtml,http://gshow.globo.com/tudo-mais/viralizou/noticia/larissa-manoela-ganhou-mais-de-2-milhoes-de-seguidores-apos-entrevista-sobre-o-rompimento-com-os-pais.ghtml,http://gshow.globo.com/novelas/relembre-a-carreira-de-larissa-manoela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/em-meio-a-polemica-de-pais-gananciosos-sandy-diz-que-xororo-sempre-a-incluiu-em-decisoes.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-mostra-novos-prints-sobre-treta-com-mulher-de-thiago-silva.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-comenta-polemicas-e-desabafa-em-live.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/belle-silva-se-pronuncia-sobre-rumores-de-thiago-participar-em-programa-de-emagrecimento.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/maira-cardi-rebate-critica-de-daniel-cady.ghtml\n
2,http://gshow.globo.com/tudo-mais/pop/noticia/dublador-de-mario-se-aposenta-de-personagem-apos-mais-de-30-anos.ghtml,Dublador de Mario se aposenta de personagem após mais de 30 anos,http://gshow.globo.com/tudo-mais/pop/noticia/matheus-nachtergaele-sobre-o-auto-da-compadecida-2-nao-sei-o-tamanho-da-emocao-que-estou-sentindo.ghtml,http://gshow.globo.com/novelas/forca-de-um-desejo-figurino-inspirado-em-imperatriz-e-personagem-revisitada-confira-outras-curiosidades.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/relembre-a-carreira-de-larissa-manoela.ghtml,http://gshow.globo.com/tudo-mais/tv-e-famosos/noticia/larissa-manoela-tem-dia-de-malhacao-com-o-noivo-andre-luiz-frambach.ghtml,http:

In [32]:
# Salvar em CSV
df_recomendacoes_gemini.to_csv("materias_recomendadas_iagen.csv", index=False)

Os resultados com IA Generativa ficaram bons. Foi o que a recomendação ficou mais assertiva, o único ponto que deve ser levado em consideração é o custo e o tempo de processamento.<br>
Para o modelo utilizado: Entrada: US$ 1,25 por milhão de tokens / Saída: US$ 5,00 por milhão de tokens<br>
Existem outros modelos mais baratos, deve-se analisar a assertividade e o custo.

<b>Diante das técnicas utilizadas os melhores resultados foram:</b>


1º IA Generativa Gemini utilizando o resumo dos conteúdos
<br>
2º NLP utilizando TF_IDF com biblioteca nltk em português e função cosseno
